## ChatBot to (Chat with Pdfs) 🤖

In [ ]:
from langchain_openai import  OpenAI 
from langchain_community.document_loaders import  TextLoader
from langchain.text_splitter import  RecursiveCharacterTextSplitter
from langchain_openai import  OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.vectorstores import  VectorStoreRetriever 
from langchain.chains import RetrievalQA 
from langchain.prompts import PromptTemplate
import os 
from dotenv import  load_dotenv
from langchain_groq import ChatGroq 
load_dotenv()

True

In [31]:
groq_api_key= os.getenv('GROQ_CLOUD_API')

In [32]:
model_name = "llama-3.1-70b-versatile"
llm = ChatGroq(temperature=0.7,
                groq_api_key=groq_api_key,
                model_name=model_name)

In [6]:
response = llm.invoke("The first person land on moon was?")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [ ]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path):

    try:
        # Open the PDF file
        pdf_reader = PdfReader(pdf_path)
        
        # Extract text from each page
        extracted_text = ""
        for page in pdf_reader.pages:
            extracted_text += page.extract_text()
        
        print("Text extraction successful!")
        return extracted_text
    except Exception as e:
        print(f"Error: {e}")
        return ""
pdf_files = ["pdf1.pdf","pdf2.pdf"]


Text extraction successful!
Text extraction successful!


In [29]:

def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=30)
    chunks = text_splitter.split_text(text)
    print("splitted text into chunks!")
    return chunks

def get_vector_store(text_chunks):
    embedings = OpenAIEmbeddings(api_key=os.getenv("gpt_api_key"))
    vector_store = FAISS.from_texts(text_chunks, embedding=embedings)
    vector_store.save_local("faiss_index")
    return vector_store



text = ""
for path in pdf_files:
    raw_text = extract_text_from_pdf(pdf_path=path)
    text += raw_text

chunks = get_text_chunks(text) 
vector_store = get_vector_store(chunks)

Text extraction successful!
Text extraction successful!
splitted text into chunks!


In [30]:
query_answer = vector_store.similarity_search("This revolutionary new breakthrough in AF technology ")
page_content = query_answer[0].page_content

# Split the content at "Answer:" and get the part after it
# answer = page_content.split("Answer:")[1].strip()

print(page_content)


A camera that sees and understands your world. With a new, evolved EOS iTR (Intelligent Tracking & Recognition)
AF X using deep learning technology, subject detection of the EOS R3 is faster and smarter than ever.
New Vehicle Priority AF
This revolutionary new breakthrough in AF technology now allows for detection and recognition of fast-moving
automobiles and motorcycles in motorsports. The EOS R3 is capable of overall detection of the vehicle and also
more speci c focus of the driver’s helmet using spot detection.
Enhanced Eye Detection AF
Performance
Subject detection precision has also been
enhanced for humans.
Eye Detection AF is possible even when the
subject’s face is obscured by shadow, hair,
make-up or a face mask.  Head detection is alsoAnimal Detection AF
First introduced in the EOS R5, Animal
Detection AF is also available on the EOS R3,
allowing for detection and tracking of the body,
face or eye of dogs, cats and birds.
possible for athletes wearing goggles or face


In [33]:
retriever = vector_store.as_retriever(score_threshold=0.8)
query_chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff',retriever=retriever)
query = "Tell me about EOS R3 specifications."
response = query_chain.invoke(query)
print(response['result'])

I can provide some information about the EOS R3 specifications based on the given context. Here are a few:

1. Image Sensor: Approximately 24.1-megapixel back-illuminated stacked CMOS sensor
2. Image Processor: DIGIC X
3. Autofocus: Eye Control AF + New Vehicle Priority AF
4. Continuous Shooting: Up to 30 fps with AF/AE tracking
5. Video Recording: 6K RAW recording + 4K movie with 6K oversampling
6. Electronic Viewfinder (EVF): Approximately 5.76-million-dot 1.27cm (0.5in) OLED EVF with up to 120 fps refresh rate
7. LCD Screen: 4.15-million-dot 8.1cm (3.2in) high-definition articulating touchscreen LCD
8. Shutter: Highly durable shutter capable of shooting up to 500,000 cycles, with silent shutter mode
9. Battery: Long-lasting high-capacity lithium-ion battery (same as the EOS-1D X series)
10. Image Stabilization: 5-axis in-body image stabilization (IS), with up to 8 stops of image stabilization possible when combined with compatible RF lenses
11. ISO Sensitivity: Normal ISO speed rang

In [ ]:
def get_qa_chain(retriever):

    prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """

    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context","question"]
    )

    chain = RetrievalQA.from_chain_type(llm=llm,
                                        chain_type="stuff",
                                        retriever=retriever,
                                        input_key="query",
                                        return_source_documents=True,
                                        chain_type_kwargs={"prompt": PROMPT})

    return chain


#  QUESTION 1:
chain = get_qa_chain(retriever) 
print(chain("How we can print hello world?")['result'])


C:\Users\Ranjit\AppData\Local\Temp\ipykernel_8932\2106802554.py:27: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(chain("How we can print hello world?")['result'])


Answer is not available in the context.


In [ ]:
#  QUESTION 2:
print(chain("who is the prime minister of india?")['result'])

Answer is not available in the context


In [ ]:
#  QUESTION 3:
print(chain("Tell me about something Superior Image Quality of EOS r3")['result'])

The EOS R3 is equipped with a new approximately 24.1-megapixel back-illuminated stacked CMOS sensor combined with DIGIC X image processor for the first time, allowing for high-speed readout and high image quality with low noise.


In [ ]:
#  QUESTION 4:
print(chain("Powerful In-Body Image Stabilisation")['result'])

The EOS R3 is equipped with 5-axis in-body image stabilisation (IS). This feature suppresses blurriness caused by camera shake, increasing the possibilities for handheld shooting, low-light shooting, and long exposure shooting. When combined with the optical image stabilizer of compatible RF lenses, up to 8 stops of image stabilisation can be achieved via Coordinated Control IS.


In [39]:
#  You can write your question -----
question = input("write your question : ")
print("User 🙂 :",question)
print("Boat 🤖 : ",chain(question)['result'])


User 🙂 : what level of High ISO Sensitivity
Boat 🤖 :  The EOS R3 has a wide normal ISO speed range of 100 to 102,400 (expandable to L: 50 and H: 204,800), allowing for clean low-noise images even in low-light situations.


### ✨ Thank you for giving me the opportunity to prove myself. I truly appreciate it! 🙏